In [1]:
# DART에서 삼성생명 자료 다운로드 코드

import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import os
import time

# DART API 설정

api_key = "c8d2b413072cd926b74494023c38f0ae2b2c21f9"
DART_API_KEY = "c8d2b413072cd926b74494023c38f0ae2b2c21f9"  # https://opendart.fss.or.kr/uss/umt/EgovMberInsttRegistView.do에서 발급
BASE_URL = "https://opendart.fss.or.kr/api"

def get_corp_code():
    """기업코드 조회 - 삼성생명 찾기"""
    url = f"{BASE_URL}/corpCode.xml?crtfc_key={DART_API_KEY}"

    try:
        response = requests.get(url)
        # XML 파일을 저장하고 압축 해제해야 함
        with open("corp_code.zip", "wb") as f:
            f.write(response.content)

        import zipfile
        with zipfile.ZipFile("corp_code.zip", 'r') as zip_ref:
            zip_ref.extractall(".")

        # XML 파싱하여 삼성생명 코드 찾기
        import xml.etree.ElementTree as ET
        tree = ET.parse("CORPCODE.xml")
        root = tree.getroot()

        for corp in root.findall('.//list'):
            corp_name = corp.find('corp_name').text if corp.find('corp_name') is not None else ""
            if "삼성생명" in corp_name:
                corp_code = corp.find('corp_code').text
                print(f"삼성생명 기업코드: {corp_code}")
                return corp_code

        print("삼성생명을 찾을 수 없습니다.")
        return None

    except Exception as e:
        print(f"기업코드 조회 오류: {e}")
        return None

def search_samsung_life_reports(corp_code, start_date="20230101", end_date="20241231"):
    """삼성생명 공시 목록 조회"""
    url = f"{BASE_URL}/list.json"

    params = {
        'crtfc_key': DART_API_KEY,
        'corp_code': corp_code,
        'bgn_de': start_date,
        'end_de': end_date,
        'page_no': 1,
        'page_count': 100
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()

        if data['status'] == '000':
            reports = data['list']
            print(f"삼성생명 공시 {len(reports)}건 발견")
            return reports
        else:
            print(f"공시 조회 실패: {data['message']}")
            return []

    except Exception as e:
        print(f"공시 목록 조회 오류: {e}")
        return []

def download_samsung_life_reports(reports, download_folder="samsung_life_reports"):
    """삼성생명 공시자료 다운로드"""

    # 다운로드 폴더 생성
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    downloaded_count = 0

    for i, report in enumerate(reports):
        try:
            rcept_no = report['rcept_no']
            corp_name = report['corp_name']
            report_nm = report['report_nm']
            flr_nm = report['flr_nm']
            rcept_dt = report['rcept_dt']

            print(f"[{i+1}/{len(reports)}] 다운로드 중: {report_nm}")

            # 공시서류 원본 다운로드
            download_url = f"{BASE_URL}/document.xml"
            params = {
                'crtfc_key': DART_API_KEY,
                'rcept_no': rcept_no
            }

            response = requests.get(download_url, params=params)

            if response.status_code == 200:
                # 파일명 생성 (특수문자 제거)
                safe_filename = f"{rcept_dt}_{report_nm}_{rcept_no}".replace("/", "_").replace("\\", "_").replace(":", "_")
                file_path = os.path.join(download_folder, f"{safe_filename}.xml")

                with open(file_path, 'wb') as f:
                    f.write(response.content)

                downloaded_count += 1
                print(f"✅ 저장완료: {file_path}")
            else:
                print(f"❌ 다운로드 실패: {report_nm}")

            # API 호출 제한 방지 (초당 10회 제한)
            time.sleep(0.1)

        except Exception as e:
            print(f"❌ 다운로드 오류: {e}")
            continue

    print(f"\n총 {downloaded_count}건 다운로드 완료!")
    return downloaded_count

def get_samsung_life_financial_data(corp_code, bsns_year="2023", reprt_code="11013"):
    """삼성생명 재무제표 데이터 조회"""

    # reprt_code: 11013=1분기, 11012=반기, 11014=3분기, 11011=사업보고서
    url = f"{BASE_URL}/fnlttSinglAcnt.json"

    params = {
        'crtfc_key': DART_API_KEY,
        'corp_code': corp_code,
        'bsns_year': bsns_year,
        'reprt_code': reprt_code
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()

        if data['status'] == '000':
            financial_data = data['list']

            # DataFrame으로 변환
            df = pd.DataFrame(financial_data)

            # 엑셀 저장
            filename = f"삼성생명_재무제표_{bsns_year}_{reprt_code}.xlsx"
            df.to_excel(filename, index=False)
            print(f"재무제표 저장: {filename}")

            return df
        else:
            print(f"재무제표 조회 실패: {data['message']}")
            return None

    except Exception as e:
        print(f"재무제표 조회 오류: {e}")
        return None

def main():
    print("=== DART 삼성생명 자료 다운로드 ===\n")

    if not DART_API_KEY or DART_API_KEY == "발급받은_API_키":
        print("❌ DART API 키를 먼저 발급받아 설정해주세요!")
        print("발급 사이트: https://opendart.fss.or.kr/uss/umt/EgovMberInsttRegistView.do")
        return

    # 1. 삼성생명 기업코드 조회
    print("1. 삼성생명 기업코드 조회 중...")
    corp_code = get_corp_code()

    if not corp_code:
        print("삼성생명 기업코드를 찾을 수 없습니다.")
        return

    # 2. 삼성생명 공시 목록 조회
    print("\n2. 삼성생명 공시 목록 조회 중...")
    reports = search_samsung_life_reports(corp_code)

    if not reports:
        print("공시 목록이 없습니다.")
        return

    # 공시 목록 미리보기
    print("\n📋 삼성생명 주요 공시 목록:")
    for i, report in enumerate(reports[:10]):  # 최근 10건만 표시
        print(f"{i+1}. [{report['rcept_dt']}] {report['report_nm']}")

    # 3. 사용자 선택
    choice = input(f"\n다운로드 옵션을 선택하세요:\n1. 전체 다운로드 ({len(reports)}건)\n2. 최근 10건만 다운로드\n3. 재무제표만 조회\n선택 (1-3): ")

    if choice == "1":
        print("\n3. 전체 공시자료 다운로드 중...")
        download_samsung_life_reports(reports)

    elif choice == "2":
        print("\n3. 최근 10건 공시자료 다운로드 중...")
        download_samsung_life_reports(reports[:10])

    elif choice == "3":
        print("\n3. 삼성생명 재무제표 조회 중...")

        # 2023년 사업보고서 재무제표
        df_annual = get_samsung_life_financial_data(corp_code, "2023", "11011")

        # 2024년 3분기 재무제표
        df_q3 = get_samsung_life_financial_data(corp_code, "2024", "11014")

        if df_annual is not None:
            print("\n📊 2023년 사업보고서 주요 재무 데이터:")
            print(df_annual[['account_nm', 'thstrm_amount']].head(10))

    else:
        print("잘못된 선택입니다.")
        return

    print("\n🎉 작업 완료!")

# GPT 활용 간단 버전
def simple_samsung_life_download():
    """GPT에게 요청할 수 있는 간단 버전"""

    # 삼성생명 고정 기업코드 (실제로는 동적 조회 필요)
    samsung_corp_code = "00123456"  # 실제 코드로 교체 필요

    # 간단한 재무제표 조회
    url = f"https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"
    params = {
        'crtfc_key': DART_API_KEY,
        'corp_code': samsung_corp_code,
        'bsns_year': '2023',
        'reprt_code': '11011'
    }

    response = requests.get(url, params=params)
    data = response.json()

    if data['status'] == '000':
        df = pd.DataFrame(data['list'])
        df.to_excel('삼성생명_재무제표.xlsx', index=False)
        print("삼성생명 재무제표 다운로드 완료!")
    else:
        print(f"오류: {data['message']}")

if __name__ == "__main__":
    main()

# GPT 활용 팁
"""
GPT에게 이렇게 요청하세요:

"DART API를 사용해서 삼성생명의 최근 3년간 재무제표를 자동으로 다운로드하는 파이썬 코드를 만들어주세요.
엑셀 파일로 저장하고, 주요 재무지표(자산, 부채, 자본, 매출, 순이익)만 추출해서 요약 리포트도 만들어주세요."

이렇게 요청하면 더 상세하고 맞춤형 코드를 받을 수 있습니다.
"""

=== DART 삼성생명 자료 다운로드 ===

1. 삼성생명 기업코드 조회 중...
삼성생명 기업코드: 00218450

2. 삼성생명 공시 목록 조회 중...
공시 조회 실패: 조회된 데이타가 없습니다.
공시 목록이 없습니다.


'\nGPT에게 이렇게 요청하세요:\n\n"DART API를 사용해서 삼성생명의 최근 3년간 재무제표를 자동으로 다운로드하는 파이썬 코드를 만들어주세요. \n엑셀 파일로 저장하고, 주요 재무지표(자산, 부채, 자본, 매출, 순이익)만 추출해서 요약 리포트도 만들어주세요."\n\n이렇게 요청하면 더 상세하고 맞춤형 코드를 받을 수 있습니다.\n'

In [2]:
# 삼성화재 DART 자료 다운로드 - 초간단 버전 (for문, 함수 없이)

import requests
import pandas as pd

# 설정
DART_API_KEY ="c8d2b413072cd926b74494023c38f0ae2b2c21f9"  # https://opendart.fss.or.kr/에서 발급
SAMSUNG_FIRE_CODE = "00126380"  # 삼성화재 기업코드

print("=== 삼성화재 DART 자료 다운로드 ===")

# 1. 삼성화재 2024년 3분기 재무제표 다운로드
print("1. 삼성화재 2024년 3분기 재무제표 조회 중...")

url1 = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"
params1 = {
    'crtfc_key': DART_API_KEY,
    'corp_code': SAMSUNG_FIRE_CODE,
    'bsns_year': '2024',
    'reprt_code': '11014'  # 3분기 보고서
}

response1 = requests.get(url1, params=params1)
data1 = response1.json()

if data1['status'] == '000':
    df1 = pd.DataFrame(data1['list'])
    df1.to_excel('삼성화재_2024년_3분기_재무제표.xlsx', index=False)
    print("✅ 2024년 3분기 재무제표 저장 완료!")
    print(f"   - 항목 수: {len(df1)}개")
else:
    print(f"❌ 오류: {data1['message']}")

# 2. 삼성화재 2023년 사업보고서 재무제표 다운로드
print("\n2. 삼성화재 2023년 사업보고서 재무제표 조회 중...")

params2 = {
    'crtfc_key': DART_API_KEY,
    'corp_code': SAMSUNG_FIRE_CODE,
    'bsns_year': '2023',
    'reprt_code': '11011'  # 사업보고서
}

response2 = requests.get(url1, params=params2)
data2 = response2.json()

if data2['status'] == '000':
    df2 = pd.DataFrame(data2['list'])
    df2.to_excel('삼성화재_2023년_사업보고서_재무제표.xlsx', index=False)
    print("✅ 2023년 사업보고서 재무제표 저장 완료!")
    print(f"   - 항목 수: {len(df2)}개")
else:
    print(f"❌ 오류: {data2['message']}")

# 3. 삼성화재 최근 공시 목록 조회
print("\n3. 삼성화재 최근 공시 목록 조회 중...")

url3 = "https://opendart.fss.or.kr/api/list.json"
params3 = {
    'crtfc_key': DART_API_KEY,
    'corp_code': SAMSUNG_FIRE_CODE,
    'bgn_de': '20240101',
    'end_de': '20241231',
    'page_no': 1,
    'page_count': 10  # 최근 10건만
}

response3 = requests.get(url3, params=params3)
data3 = response3.json()

if data3['status'] == '000':
    df3 = pd.DataFrame(data3['list'])
    df3.to_excel('삼성화재_2024년_공시목록.xlsx', index=False)
    print("✅ 2024년 공시목록 저장 완료!")
    print(f"   - 공시 건수: {len(df3)}건")

    # 주요 공시 미리보기
    print("\n📋 최근 공시 목록:")
    if len(df3) > 0:
        print(f"   최신: [{df3.iloc[0]['rcept_dt']}] {df3.iloc[0]['report_nm']}")
    if len(df3) > 1:
        print(f"   2번째: [{df3.iloc[1]['rcept_dt']}] {df3.iloc[1]['report_nm']}")
    if len(df3) > 2:
        print(f"   3번째: [{df3.iloc[2]['rcept_dt']}] {df3.iloc[2]['report_nm']}")
else:
    print(f"❌ 오류: {data3['message']}")

# 4. 삼성화재 주요 재무 지표 요약
print("\n4. 삼성화재 주요 재무지표 요약")

if 'df2' in locals() and len(df2) > 0:
    print("\n💰 2023년 주요 재무 현황:")

    # 자산총계 찾기
    asset_row = df2[df2['account_nm'].str.contains('자산총계', na=False)]
    if len(asset_row) > 0:
        asset_amount = asset_row.iloc[0]['thstrm_amount']
        print(f"   자산총계: {asset_amount}")

    # 부채총계 찾기
    debt_row = df2[df2['account_nm'].str.contains('부채총계', na=False)]
    if len(debt_row) > 0:
        debt_amount = debt_row.iloc[0]['thstrm_amount']
        print(f"   부채총계: {debt_amount}")

    # 자본총계 찾기
    equity_row = df2[df2['account_nm'].str.contains('자본총계|자본총액', na=False)]
    if len(equity_row) > 0:
        equity_amount = equity_row.iloc[0]['thstrm_amount']
        print(f"   자본총계: {equity_amount}")

    # 매출액 찾기
    sales_row = df2[df2['account_nm'].str.contains('매출액|영업수익', na=False)]
    if len(sales_row) > 0:
        sales_amount = sales_row.iloc[0]['thstrm_amount']
        print(f"   매출액: {sales_amount}")

# 5. 삼성화재 배당 정보 조회
print("\n5. 삼성화재 배당 정보 조회 중...")

url5 = "https://opendart.fss.or.kr/api/alotMatter.json"
params5 = {
    'crtfc_key': DART_API_KEY,
    'corp_code': SAMSUNG_FIRE_CODE,
    'bsns_year': '2023',
    'reprt_code': '11011'
}

response5 = requests.get(url5, params=params5)
data5 = response5.json()

if data5['status'] == '000':
    df5 = pd.DataFrame(data5['list'])
    df5.to_excel('삼성화재_배당정보.xlsx', index=False)
    print("✅ 배당정보 저장 완료!")

    if len(df5) > 0:
        print(f"   배당 정보: {len(df5)}건")
else:
    print(f"❌ 배당정보 조회 실패: {data5.get('message', '알 수 없는 오류')}")

# 6. 결과 요약
print("\n" + "="*50)
print("🎉 삼성화재 자료 다운로드 완료!")
print("="*50)
print("📁 생성된 파일들:")
print("   ✓ 삼성화재_2024년_3분기_재무제표.xlsx")
print("   ✓ 삼성화재_2023년_사업보고서_재무제표.xlsx")
print("   ✓ 삼성화재_2024년_공시목록.xlsx")
print("   ✓ 삼성화재_배당정보.xlsx")
print("\n💡 재보험 업무 활용 팁:")
print("   - 재무제표로 삼성화재 재무 건전성 분석")
print("   - 공시목록으로 최신 경영 현황 파악")
print("   - 배당정보로 투자 수익률 분석")

# 간단한 사용법 안내
print("\n" + "="*50)
print("📖 사용법:")
print("1. DART API 키를 발급받아 위의 'DART_API_KEY'에 입력")
print("2. 필요한 라이브러리 설치: pip install requests pandas openpyxl")
print("3. 코드 실행하면 자동으로 엑셀 파일 생성")
print("="*50)

# GPT에게 요청할 수 있는 추가 개선 아이디어
print("\n💻 GPT 활용 팁:")
print("다음과 같이 GPT에게 요청해보세요:")
print("'삼성화재 재무제표를 분석해서 ROE, ROA 등 주요 재무비율을 계산하고")
print(" 시각화 차트를 만드는 코드를 추가해주세요.'")

=== 삼성화재 DART 자료 다운로드 ===
1. 삼성화재 2024년 3분기 재무제표 조회 중...
✅ 2024년 3분기 재무제표 저장 완료!
   - 항목 수: 30개

2. 삼성화재 2023년 사업보고서 재무제표 조회 중...
✅ 2023년 사업보고서 재무제표 저장 완료!
   - 항목 수: 30개

3. 삼성화재 최근 공시 목록 조회 중...
✅ 2024년 공시목록 저장 완료!
   - 공시 건수: 10건

📋 최근 공시 목록:
   최신: [20241223] 임원ㆍ주요주주특정증권등소유상황보고서
   2번째: [20241223] [기재정정]기타경영사항(자율공시)              
   3번째: [20241220] 주식등의대량보유상황보고서(일반)

4. 삼성화재 주요 재무지표 요약

💰 2023년 주요 재무 현황:
   자산총계: 455,905,980,000,000
   부채총계: 92,228,115,000,000
   자본총계: 363,677,865,000,000
   매출액: 258,935,494,000,000

5. 삼성화재 배당 정보 조회 중...
✅ 배당정보 저장 완료!
   배당 정보: 15건

🎉 삼성화재 자료 다운로드 완료!
📁 생성된 파일들:
   ✓ 삼성화재_2024년_3분기_재무제표.xlsx
   ✓ 삼성화재_2023년_사업보고서_재무제표.xlsx
   ✓ 삼성화재_2024년_공시목록.xlsx
   ✓ 삼성화재_배당정보.xlsx

💡 재보험 업무 활용 팁:
   - 재무제표로 삼성화재 재무 건전성 분석
   - 공시목록으로 최신 경영 현황 파악
   - 배당정보로 투자 수익률 분석

📖 사용법:
1. DART API 키를 발급받아 위의 'DART_API_KEY'에 입력
2. 필요한 라이브러리 설치: pip install requests pandas openpyxl
3. 코드 실행하면 자동으로 엑셀 파일 생성

💻 GPT 활용 팁:
다음과 같이 GPT에게 요청해보세요:
'삼성화재 재무제표를 분석해서 ROE, ROA 등 주요 재무비